#### Toda la información de este documento está obtenida de [aquí](http://gpd.sip.ucm.es/rafa/docencia/nosql/Agregando.html)

# Agregaciones

## Primero vamos a añadir una serie de datos a base de datos pruebas 

In [ ]:
use pruebas
db.sesiones.insert({nombre:"Bertoldo", mes:"Marzo", distKm:6, tiempoMin:42})
db.sesiones.insert({nombre:"Herminia", mes:"Marzo", distKm:10, tiempoMin:60})
db.sesiones.insert({nombre:"Bertoldo", mes:"Marzo", distKm:2, tiempoMin:12})
db.sesiones.insert({nombre:"Herminia", mes:"Marzo", distKm:10, tiempoMin:61})
db.sesiones.insert({nombre:"Bertoldo", mes:"Abril", distKm:5, tiempoMin:33})
db.sesiones.insert({nombre:"Herminia", mes:"Abril", distKm:42, tiempoMin:285})
db.sesiones.insert({nombre:"Aniceto", mes:"Abril", distKm:5, tiempoMin:33})


## Supongamos que queremos saber el número de sesiones que ha realizado cada persona:


In [ ]:
db.sesiones.aggregate(                                  
                      [                                 
                       {$group:                         
                               { _id:"$nombre",         
                                 num_sesiones:          
                                              {$sum:1}  
                               }
                        }
                       ]
                     )

##También se pueden agrupar por más de una clave
db.sesiones.aggregate(
                    [
                        {$group:
                                { _id:{nombre:"$nombre",
                                  mes: "$mes"},
                                  num_sesiones: {$sum:1}
                                }
                        }
                    ]
                    )




<font color=blue>sum </font>: suma (o incrementa) <br>
<font color=blue>avg</font>: mínimo de los valores<br>
<font color=blue>min</font>: máximo<br>
<font color=blue>max</font>: suma (o incrementa)<br>
<font color=blue>push</font>:  Mete en un array un valor determinado<br>
<font color=blue>addToSet</font>:  Mete en un array los valores que digamos, pero solo una vez<br>
<font color=blue>first</font>: obtiene el primer elemento del grupo, a menudo junto con sort<br>
<font color=blue>last</font>: obtiene el último elemento, a menudo junto con sort<br>


## addToSet

 crea arrays agrupando elementos.

In [ ]:
db.sesiones.aggregate(
                      [
                       {$group:
                               { _id:{nombre:"$nombre"},
                                 distancias: {$addToSet:'$distKm'}
                               }
                        }
                       ]
                     )

## unwind

Es el reverso de <font color=blue>push</font>.<br>
Cuando tenemos documentos que contienen un array y queremos agrupar por valores del array, a veces conviene eliminar los arrays y convertirlos en múltiples documentos.
## Queremos saber el número de personas con el que cuenta cada afición. ¿Cómo hacerlo?


In [ ]:

db.gustos.insert({nombre:"Bertoldo",  aficiones:["siesta","cine"]})
db.gustos.insert({nombre:"Herminia", aficiones:["correr","cine"]})
db.gustos.insert({nombre:"Aniceta", aficiones:["viajar","cine"]})
db.gustos.insert({nombre:"Godofredo",  aficiones:["correr","montaña", "cine"]})

db.gustos.aggregate([   {$unwind:'$aficiones'}   ] )

#Resultado ------------
{ "_id" : ObjectId("56b513cc5df4eba0d451ffaa"), "nombre" : "Bertoldo", "aficiones" : "siesta" }
{ "_id" : ObjectId("56b513cc5df4eba0d451ffaa"), "nombre" : "Bertoldo", "aficiones" : "cine" }
{ "_id" : ObjectId("56b513cc5df4eba0d451ffab"), "nombre" : "Herminia", "aficiones" : "correr" }
{ "_id" : ObjectId("56b513cc5df4eba0d451ffab"), "nombre" : "Herminia", "aficiones" : "cine" }
{ "_id" : ObjectId("56b513cc5df4eba0d451ffac"), "nombre" : "Aniceta", "aficiones" : "viajar" }
{ "_id" : ObjectId("56b513cc5df4eba0d451ffac"), "nombre" : "Aniceta", "aficiones" : "cine" }
{ "_id" : ObjectId("56b513cd5df4eba0d451ffad"), "nombre" : "Godofredo", "aficiones" : "correr" }
{ "_id" : ObjectId("56b513cd5df4eba0d451ffad"), "nombre" : "Godofredo", "aficiones" : "montaña" }
{ "_id" : ObjectId("56b513cd5df4eba0d451ffad"), "nombre" : "Godofredo", "aficiones" : "cine" }




### Una vez separado en aficiones ya solo queda agrupar por aficcion y contar

In [ ]:
db.gustos.aggregate([
                     {$unwind:'$aficiones'},
                     {$group:
                          {_id:'$aficiones',
                           total:{$sum:1} } }
                    ] )


#Resultado
{ "_id" : "montaña", "total" : 1 }
{ "_id" : "viajar", "total" : 1 }
{ "_id" : "correr", "total" : 2 }
{ "_id" : "cine", "total" : 4 }
{ "_id" : "siesta", "total" : 1 }
 

## match
Sería el <font color=Red>where</font> equivalente en SQL

In [ ]:
db.sesiones.aggregate( [
   {$group: { _id:{nombre:"$nombre", mes: "$mes"}, media: $avg:'$distKm'} } },
   {$match: {media:{$gt:5}} } ]
)

#Resultado
{ "_id" : { "nombre" : "Herminia", "mes" : "Abril" }, "media" : 42 }
{ "_id" : { "nombre" : "Herminia", "mes" : "Marzo" }, "media" : 10 }
 